In [3]:
import pandas as pd
import time
import probabilitylib as pl
from probabilitylib import ProbabilitySpace
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import random
np.random.seed(42)
random.seed(42)

In [ ]:
#create a function to generate a synthetic dataset
def generate_customer_data(num_records):
    data = []

    def maybe_nan(value):
        return value if random.random() > 0.2 else np.nan  

    for _ in range(num_records):
        age = np.random.randint(18, 80)
       
        # Rule: Income is loosely based on age
        base_income = 20000 + (age - 18) * 1000
        income = np.random.normal(base_income, base_income * 0.2)
       
        # Rule: Credit score is influenced by age and income
        credit_score = min(850, max(300, int(600 + (age/80)*100 + (income/100000)*100 + np.random.normal(0, 50))))
       
        # Rule: Loan amount is based on income and credit score
        max_loan = income * (credit_score / 600)
        loan_amount = np.random.uniform(0, max_loan)

        gender = np.random.choice(['Male', 'Female'])

        if age <= 30 and gender == "Female":
            smoker_prob = 0.1
        elif age <= 30 and gender == "Male":
            smoker_prob = 0.05
        elif age in range(31, 50) and gender == "Female":
            smoker_prob = 0.35
        elif age in range(31, 50) and gender == "Male":
            smoker_prob = 0.25
        elif age >= 51 and gender == "Female":
            smoker_prob = 0.6
        else:
            smoker_prob = 0.5
            
        smoker = random.choices([True, False], weights=[smoker_prob, 1 - smoker_prob])[0]

        weights = np.random.random()

    
        row = [
            maybe_nan(age),
            maybe_nan(income),
            maybe_nan(credit_score),
            maybe_nan(loan_amount),
            maybe_nan(gender),
            maybe_nan(smoker),
            maybe_nan(weights)
        ]
        data.append(row)

    return pd.DataFrame(data, columns=['Age', 'Income', 'CreditScore', 'LoanAmount', 'Gender', 'Smoker', 'weights'])

#create a synthetic dataset with 500000 rows. 
df = generate_customer_data(500000)

#print the first 5 rows of the dataset and its length
print(df.head())
print(len(df))

In [ ]:
import csv
#create a binary version of gender
df["GenderNumeric"] = df["Gender"].map({"Male": 0, "Female": 1})

#convert df to a probability space
ps = ProbabilitySpace(df)

#convert df to a csv file
d = df.to_csv("example.csv", index=False)
print("CSV file 'example.csv' created successfully.")


In [ ]:
#find the average percentage error between expected mean and combine_expect_mean

errors_expected_mean = []
chunk_sizes_expected_mean = []
variables = []


for i in range(100):

    var = random.choice(["Age", "Income", "CreditScore", "LoanAmount", "GenderNumeric"])
    variables.append(var)

    res = pl.expected_mean(ps, var)
    
    # Random chunk size
    m = np.random.randint(1000, 250000)
    chunk_sizes_expected_mean.append((m/500000)*100)
    
    
    chunk_results = pl.process_in_chunks(
        "example.csv", m, 
        pl.expected_mean, 
        var
    )
    
    # Combine counts to get global MI
    combined_expected_mean = pl.combine_means(chunk_results)
    
    # Compute % error
    error = abs(res - combined_expected_mean) / ((res + combined_expected_mean) / 2) * 100
    errors_expected_mean.append(error)
    
    print(f"Iteration {i} completed, chunk size={m}, error={error:.5f}%")


In [5]:
#print the average error and the minimum and amximum errors
print(f"📊 Average % error: {np.mean(errors_expected_mean):.15f}%")
print(f"📉 Min error: {np.min(errors_expected_mean):.15f}%, Max error: {np.max(errors_expected_mean):.15f}%")

📊 Average % error: 0.000148797903251%
📉 Min error: 0.000000679097799%, Max error: 0.001095873825836%


In [6]:
#create DataFrame with columns containing the variables, chunk sizes selected, and the percentage error
df_results = pd.DataFrame({
    "variable": variables,
    "chunk_size": chunk_sizes_expected_mean,
    "error_percent": errors_expected_mean
})

#return a statistical breakdown of the DataFrame
summary = df_results.groupby("variable")["error_percent"].describe()
print(summary)

               count      mean       std           min       25%       50%  \
variable                                                                     
Age             23.0  0.000121  0.000114  2.279881e-06  0.000052  0.000110   
CreditScore     19.0  0.000013  0.000020  6.790978e-07  0.000002  0.000008   
GenderNumeric   17.0  0.000230  0.000275  2.178752e-05  0.000065  0.000138   
Income          20.0  0.000164  0.000168  2.724411e-05  0.000065  0.000089   
LoanAmount      21.0  0.000221  0.000188  2.768506e-06  0.000124  0.000185   

                    75%       max  
variable                           
Age            0.000139  0.000442  
CreditScore    0.000014  0.000083  
GenderNumeric  0.000207  0.001096  
Income         0.000171  0.000644  
LoanAmount     0.000261  0.000764  


In [ ]:
#compute average percentage error per variable
avg_errors = df_results.groupby("variable")["error_percent"].mean()

#bar chart of errors per variable
avg_errors.plot(kind="bar", figsize=(8,5))
plt.ylabel("Average % Error")
plt.yscale("log")
plt.title("Average Percentage Error by Variable")
plt.savefig("expected_mean_deviation_by_variable.png", dpi=300, bbox_inches='tight')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#plot the effect of sample proportion on percentage error
plt.figure(figsize=(10,6))
plt.scatter(chunk_sizes_expected_mean, errors_expected_mean, alpha=0.6)
plt.xlabel("Percentage of rows sampled")
plt.ylabel("Percentage error (%)")
plt.title("Effect of sampling proportion on percentage error for expected_mean")
plt.grid(True)
plt.savefig("chunk_error_plot_expected_mean.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#run the same process for variance
errors_variance = []
chunk_sizes_variance = []
variables = []

for i in range(100):

    var = random.choice(["Age", "Income", "CreditScore", "LoanAmount", "GenderNumeric"])
    variables.append(var)

    res = pl.variance(ps, var)
    
    # Random chunk size
    m = np.random.randint(1000, 250000)
    chunk_sizes_variance.append((m/500000)*100)
    
    
    chunk_results = pl.process_in_chunks(
        "example.csv", m, 
        pl.variance, 
        var
    )
    
    combined_variance = pl.combine_variance(chunk_results)
    
    error = abs(res - combined_variance) / ((res + combined_variance) / 2) * 100
    errors_variance.append(error)
    
    print(f"Iteration {i} completed, chunk size={m}, error={error:.5f}%")

In [10]:
print(f"📊 Average % error: {np.mean(errors_variance):.15f}%")
print(f"📉 Min error: {np.min(errors_variance):.15f}%, Max error: {np.max(errors_variance):.15f}%")

📊 Average % error: 0.000325093476322%
📉 Min error: 0.000000104985450%, Max error: 0.003269092922550%


In [11]:
df_results_variance = pd.DataFrame({
    "variable": variables,
    "chunk_size": chunk_sizes_variance,
    "error_percent": errors_variance
})

# 2. Summary statistics by variable
summary = df_results_variance.groupby("variable")["error_percent"].describe()
print(summary)

               count          mean           std           min           25%  \
variable                                                                       
Age             22.0  2.071456e-04  2.188709e-04  6.077574e-06  6.724582e-05   
CreditScore     20.0  2.713676e-04  5.211085e-04  1.366819e-05  6.871100e-05   
GenderNumeric   15.0  8.535950e-07  6.692489e-07  1.049854e-07  4.371725e-07   
Income          25.0  4.803559e-04  7.369742e-04  7.403765e-06  1.227769e-04   
LoanAmount      18.0  5.835052e-04  7.919809e-04  5.445995e-05  1.282712e-04   

                        50%       75%       max  
variable                                         
Age            1.206222e-04  0.000312  0.000922  
CreditScore    1.042582e-04  0.000232  0.002401  
GenderNumeric  5.627043e-07  0.000001  0.000002  
Income         2.269174e-04  0.000367  0.003269  
LoanAmount     2.933229e-04  0.000534  0.002908  


In [ ]:
avg_errors_variance = df_results_variance.groupby("variable")["error_percent"].mean()


avg_errors_variance.plot(kind="bar", figsize=(8,5))
plt.ylabel("Average % Error")
plt.yscale("log")
plt.title("Average Percentage Error by Variable")
plt.savefig("variance_deviation_by_variable.png", dpi=300, bbox_inches='tight')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(chunk_sizes_variance, errors_variance, alpha=0.6)
plt.xlabel("Percentage of rows sampled")
plt.ylabel("Percentage error (%)")
plt.title("Effect of sampling proportion on percentage error for variance")
plt.grid(True)
plt.savefig("chunk_error_plot_variance.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#same process for IQR

errors_IQR = []
chunk_sizes_IQR = []
sample_props_IQR = []
variables = []

np.random.seed(42)

for i in range(100):

    var = random.choice(["Age", "Income", "CreditScore", "LoanAmount"])
    variables.append(var)

    res = pl.IQR(ps, var)["IQR"]
    
    # Random chunk size
    m = np.random.randint(1000, 250000)
    x = np.random.random() * 0.7
    
    chunk_sizes_IQR.append((m/500000)*100)

    
    chunk_results = pl.process_in_chunks(
        "example.csv", m, 
        pl.IQR, 
        var,
        sample_size=int(x*m)    
    )

    sample_props_IQR.append((int(x*m)/m)*100) 
    
    combined_IQR = pl.combine_IQR(chunk_results, var)["IQR"]
    
    error = abs(res - combined_IQR) / ((res + combined_IQR) / 2) * 100
    errors_IQR.append(error)
    
    print(f"Iteration {i} completed, error={error:.5f}%")


In [15]:
print(f"📊 Average % error: {np.mean(errors_IQR):.15f}%")
print(f"📉 Min error: {np.min(errors_IQR):.15f}%, Max error: {np.max(errors_IQR):.15f}%")

📊 Average % error: 0.247525532647710%
📉 Min error: 0.000000000000000%, Max error: 2.319782785038674%


In [16]:
df_results_IQR = pd.DataFrame({
    "variable": variables,
    "chunk_size": chunk_sizes_IQR,
    "error_percent": errors_IQR,
    "sample_size": sample_props_IQR
})

summary = df_results_IQR.groupby("variable")["error_percent"].describe()
print(summary)

             count      mean       std           min       25%       50%  \
variable                                                                   
Age           19.0  0.000000  0.000000  0.000000e+00  0.000000  0.000000   
CreditScore   32.0  0.363419  0.520197  0.000000e+00  0.000000  0.000000   
Income        30.0  0.324054  0.599675  2.571831e-11  0.045336  0.135034   
LoanAmount    19.0  0.179028  0.231963  1.239035e-02  0.058896  0.090342   

                  75%       max  
variable                         
Age          0.000000  0.000000  
CreditScore  1.104972  1.117318  
Income       0.201260  2.319783  
LoanAmount   0.207094  1.052932  


In [ ]:
avg_errors_IQR = df_results_IQR.groupby("variable")["error_percent"].mean()

avg_errors_IQR.plot(kind="bar", figsize=(8,5))
plt.ylabel("Average % Error")
plt.yscale("log")
plt.title("Average Percentage Error by Variable")
plt.savefig("IQR_deviation_by_variable.png", dpi=300, bbox_inches='tight')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(chunk_sizes_IQR, errors_IQR, alpha=0.6)
plt.xlabel("chunk_size")
plt.ylabel("Percentage error (%)")
plt.title("Effect of chunking proportion on percentage error for IQR")
plt.grid(True)
plt.savefig("chunk_error_plot_IQR.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(sample_props_IQR, errors_IQR, alpha=0.6)
plt.xlabel("Percentage of rows sampled")
plt.ylabel("Percentage error (%)")
plt.title("Effect of sampling proportion on percentage error for IQR")
plt.grid(True)
plt.savefig("sampling_error_plot_IQR.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#same process for mutual information

#start with analysing mutual information with pairs of discrete variables

errors_mutual_information_discrete = []
chunk_sizes_mutual_information_discrete = []
variables = ["Gender", "GenderNumeric", "Smoker"]

np.random.seed(42)

for i in range(100):
    iter_start = time.time()
    
    x = random.choice(variables)
    y = random.choice([v for v in variables if v != x])

    res = pl.mutual_information(ps, x, y)
    
    # Random chunk size
    m = np.random.randint(1000, 250000)
    chunk_sizes_mutual_information_discrete.append((m/500000)*100)
    
    chunk_results = pl.process_in_chunks(
        "example.csv", m, 
        pl.mutual_information, 
        x,
        y   
    )
    combined_MI = pl.combine_mutual_information(chunk_results)
    
    error = abs(res - combined_MI) / ((res + combined_MI) / 2) * 100
    errors_mutual_information_discrete.append(error)
    
    print(f"Iteration {i} completed, chunk size={m}, error={error:.5f}%")



In [21]:
print(f"📊 Average % error: {np.mean(errors_mutual_information_discrete):.15f}%")
print(f"📉 Min error: {np.min(errors_mutual_information_discrete):.15f}%, Max error: {np.max(errors_mutual_information_discrete):.15f}%")

📊 Average % error: 0.000000000001760%
📉 Min error: 0.000000000000000%, Max error: 0.000000000013367%


In [ ]:
#analyse percentage error when both variables are continuous

errors_mutual_information_continuous = []
chunk_sizes_mutual_information_continuous = []
variables = ["Age", "Income", "CreditScore", "LoanAmount"]
var_list = []

for i in range(100):
   
    m = np.random.randint(1000, 250000)
    chunk_sizes_mutual_information_continuous.append((m/500000)*100)

    x = random.choice(variables)
    y = random.choice([v for v in variables if v != x])

    var_list.append([x, y])
    
    #regular function use to avoid loop having to use binning_in_chunks repeatedly
    bin_boundaries = pl.bin_continuous_columns(ps, [x, y], return_edges=True)

    res = pl.mutual_information(ps, x, y, bin_boundaries=bin_boundaries)
    
    chunk_results = pl.process_in_chunks(
        "example.csv", m, 
        pl.mutual_information, 
        x,
        y,
        bin_boundaries=bin_boundaries
    )
    
    combined_MI = pl.combine_mutual_information(chunk_results)
    
    error = abs(res - combined_MI) / ((res + combined_MI) / 2) * 100
    errors_mutual_information_continuous.append(error)
       
    print(f"Iteration {i} completed, chunk size={m}, error={error:.5f}%")

In [23]:
print(f"📊 Average % error: {np.mean(errors_mutual_information_continuous):.15f}%")
print(f"📉 Min error: {np.min(errors_mutual_information_continuous):.15f}%, Max error: {np.max(errors_mutual_information_continuous):.15f}%")

📊 Average % error: 0.000000000000137%
📉 Min error: 0.000000000000000%, Max error: 0.000000000000598%


In [ ]:
#finally find average error when one variable is continuous and one is discrete

errors_mutual_information_mixed = []
chunk_sizes_mutual_information_mixed = []
x_vars = []
y_vars = []

con_variables = ["Age", "Income", "CreditScore", "LoanAmount"]
discrete_variables = ["Gender", "GenderNumeric", "Smoker"]

for i in range(100):
    iter_start = time.time()
    
    m = np.random.randint(1000, 250000)
    chunk_sizes_mutual_information_mixed.append((m/500000)*100)

    x = random.choice(con_variables)
    y = random.choice(discrete_variables)
    x_vars.append(x)
    y_vars.append(y)

    bin_boundaries = pl.bin_continuous_columns(ps, [x], return_edges=True)

    res = pl.mutual_information(ps, x, y, bin_boundaries=bin_boundaries)

    chunk_results = pl.process_in_chunks(
        "example.csv",
        m, 
        pl.mutual_information, 
        x,
        y,
        bin_boundaries=bin_boundaries
    )
    
    combined_MI = pl.combine_mutual_information(chunk_results)
    
    error = abs(res - combined_MI) / ((res + combined_MI) / 2) * 100
    errors_mutual_information_mixed.append(error)
    
    print(f"Iteration {i} completed, chunk size={m}, error={error:.5f}%")


In [25]:
print(f"📊 Average % error: {np.mean(errors_mutual_information_mixed):.15f}%")
print(f"📉 Min error: {np.min(errors_mutual_information_mixed):.15f}%, Max error: {np.max(errors_mutual_information_mixed):.15f}%")

📊 Average % error: 0.000000004574002%
📉 Min error: 0.000000000000019%, Max error: 0.000000043958147%


In [ ]:
#same process for conditional probability. Loop contains a range of variable combinations
#ensuring both continuous and discrete variables are featured.

errors_conditional = []
chunk_sizes_conditional = []
n = []

for i in range(100):
    x = random.choice([1, 2, 3, 4, 5])
    n.append(x)
    
    m = np.random.randint(1000, 250000)
    chunk_sizes_conditional.append((m/500000)*100)

    if x==1:

        res = pl.conditional(
            ps,
            event_condition = ps.df["Smoker"] == True,
            given_condition = ps.df["GenderNumeric"] == 0
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.conditional,
                lambda df: df["Smoker"] == True,
                lambda df: df["GenderNumeric"] == 0
            )
        
        combined_conditional = pl.combine_conditional(chunk_results)

    if x==2:

        res = pl.conditional(
            ps,
            event_condition = ps.df["Smoker"] == True,
            given_condition = ps.df["Income"] > 40000
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.conditional,
                lambda df: df["Smoker"] == True,
                lambda df: df["Income"] > 40000
            )
        
        combined_conditional = pl.combine_conditional(chunk_results)

    if x==3:

        res = pl.conditional(
            ps,
            event_condition = ps.df["CreditScore"] < 650,
            given_condition = ps.df["Income"] > 40000
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.conditional,
                lambda df: df["CreditScore"] < 650,
                lambda df: df["Income"] > 40000
            )
        
        combined_conditional = pl.combine_conditional(chunk_results)

    if x==4:

        res = pl.conditional(
            ps,
            event_condition = ps.df["Age"] < 65,
            given_condition = ps.df["Income"] > 40000
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.conditional,
                lambda df: df["Age"] < 65,
                lambda df: df["Income"] > 40000
            )
        
        combined_conditional = pl.combine_conditional(chunk_results)

    if x==5:

        res = pl.conditional(
            ps,
            event_condition = ps.df["GenderNumeric"] == 0,
            given_condition = ps.df["Gender"] == "Male"
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.conditional,
                lambda df: df["GenderNumeric"] == 0,
                lambda df: df["Gender"] == "Male"
            )
        
        combined_conditional = pl.combine_conditional(chunk_results)

    
    
    error = abs(res - combined_conditional) / ((res + combined_conditional) / 2) * 100
    errors_conditional.append(error)
    
    print(f"Iteration {i} completed, chunk size={m}, error={error:.5f}%")

In [27]:
print(f"📊 Average % error: {np.mean(errors_conditional):.15f}%")
print(f"📉 Min error: {np.min(errors_conditional):.15f}%, Max error: {np.max(errors_conditional):.15f}%")

📊 Average % error: 0.000000000000022%
📉 Min error: 0.000000000000000%, Max error: 0.000000000000063%


In [ ]:
#same process for joint probability

errors_joint_probability = []
chunk_sizes_joint_probability = []
n = []

for i in range(100):
    x = random.choice([1, 2, 3, 4, 5])
    n.append(x)
    y= random.choice(["dependent", "independent"])
    
    m = np.random.randint(1000, 250000)
    chunk_sizes_joint_probability.append((m/500000)*100)

    if x==1:

        res = pl.joint_probability(
            ps,
            ps.df["Smoker"] == True,
            ps.df["GenderNumeric"] == 0,
            con=y
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.joint_probability,
                lambda df: df["Smoker"] == True,
                lambda df: df["GenderNumeric"] == 0,
                con=y
            )
        
    if x==2:

        res = pl.joint_probability(
            ps,
            ps.df["Smoker"] == True,
            ps.df["Income"] > 40000,
            con=y
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.joint_probability,
                lambda df: df["Smoker"] == True,
                lambda df: df["Income"] > 40000,
                con=y
            )
        

    if x==3:

        res = pl.joint_probability(
            ps,
            ps.df["CreditScore"] < 650,
            ps.df["Income"] > 40000,
            con=y
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.joint_probability,
                lambda df: df["CreditScore"] < 650,
                lambda df: df["Income"] > 40000,
                con=y
            )
        

    if x==4:

        res = pl.joint_probability(
            ps,
            ps.df["Age"] < 65,
            ps.df["Income"] > 40000,
            con=y
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.joint_probability,
                lambda df: df["Age"] < 65,
                lambda df: df["Income"] > 40000,
                con=y
            )
        

    if x==5:

        res = pl.joint_probability(
            ps,
            ps.df["GenderNumeric"] == 0,
            ps.df["Gender"] == "Male",
            con=y
        )
               
        chunk_results = pl.process_in_chunks(
                "example.csv",
                m,
                pl.joint_probability,
                lambda df: df["GenderNumeric"] == 0,
                lambda df: df["Gender"] == "Male",
                con=y
            )
        
    combined_joint_probability = pl.combine_joint_probability(chunk_results, con=y)

    
    
    error = abs(res - combined_joint_probability) / ((res + combined_joint_probability) / 2) * 100
    errors_joint_probability.append(error)
    
    print(f"Iteration {i} completed, chunk size={m}, error={error:.5f}%")

In [19]:
print(f"📊 Average % error: {np.mean(errors_joint_probability):.15f}%")
print(f"📉 Min error: {np.min(errors_joint_probability):.15f}%, Max error: {np.max(errors_joint_probability):.15f}%")

📊 Average % error: 0.000000000000013%
📉 Min error: 0.000000000000000%, Max error: 0.000000000000081%


In [ ]:
#same process for PDF_or_PMF (discrete variables)

errors_PDF_or_PMF = []
rel_errors_PDF_or_PMF = []
chunk_sizes_PDF_or_PMF = []

variables = ["Age", "Gender", "GenderNumeric", "Smoker"]

for i in range(100):
    discrete_var = random.choice(variables)  

    full_pmf = pl.PDF_or_PMF(ps, discrete_var, var_type="discrete", plot=False)

    m = np.random.randint(1000, 250000)
    chunk_sizes_PDF_or_PMF.append((m / 500000) * 100)
    
    results = pl.process_in_chunks(
        "example.csv",  
        m,              
        pl.PDF_or_PMF,  
        discrete_var,
        var_type="discrete",
        plot=False
    )

    combined_pmf = pl.combine_PDF_or_PMF(results, var_type="discrete", plot=False)

    #calculate absolute and relative error
    abs_error = np.mean(abs(full_pmf - combined_pmf))
    rel_error = np.mean(abs(full_pmf - combined_pmf) / ((full_pmf + combined_pmf)/2) * 100)
    
    errors_PDF_or_PMF.append(abs_error)
    rel_errors_PDF_or_PMF.append(rel_error)
    
    print(f"Iteration {i} | Variable: {discrete_var} | Chunk size: {m} | Abs Error: {abs_error:.6f} | Rel Error: {rel_error:.6f}")


In [25]:
print(f"\n📊 Average % error (discrete): {np.mean(rel_errors_PDF_or_PMF):.10f}%")
print(f"📉 Min error: {np.min(rel_errors_PDF_or_PMF):.10f}%, Max error: {np.max(rel_errors_PDF_or_PMF):.10f}%")


📊 Average % error (discrete): 0.0624379191%
📉 Min error: 0.0001791912%, Max error: 0.3309506726%


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(chunk_sizes_PDF_or_PMF, rel_errors_PDF_or_PMF, alpha=0.6)
plt.xlabel("chunk_size")
plt.ylabel("Percentage error (%)")
plt.title("Effect of chunking proportion on percentage error for PDF_or_PMF (discrete variables)")
plt.grid(True)
plt.savefig("chunk_error_plot_PDF_or_PMF(discrete).png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#same process for PDF_or_PMF (continuous variables)

n_samples = 500000
var_type = "continuous"
errors = []
iteration_times = []
chunk_sizes = []
var_list = []
variables = ["Age", "Income", "LoanAmount", "CreditScore"]

for i in range(100):
    iter_start = time.time()

    variable = random.choice(variables)

    var_list.append(variable)

    full_grid, full_pdf = pl.PDF_or_PMF(ps, variable, var_type=var_type, plot=False)

    sd = pl.variance(ps, variable)**0.5
    iqr = pl.IQR(ps, variable)["IQR"] 
    global_min = ps.df[variable].min()
    global_max = ps.df[variable].max() 
    
    m = np.random.randint(1000, 250000)
    chunk_sizes.append(m)
    
    results = pl.process_in_chunks(
        "example.csv",
        m,
        pl.PDF_or_PMF,
        variable,
        var_type=var_type,
        sd=sd,
        iqr=iqr,
        n_samples=n_samples,
        global_min=global_min,
        global_max=global_max,
        grid_size=500,
        plot=False
    )
    
    combined_grid, combined_pdf = pl.combine_PDF_or_PMF(results, var_type=var_type, plot=False)
    print(np.allclose(full_grid, combined_grid))
    
    # Compare PDFs directly
    error = np.mean(abs(full_pdf - combined_pdf) / ((full_pdf + combined_pdf) / 2) * 100)
    errors.append(error)
    
    iter_end = time.time()
    iteration_times.append(iter_end - iter_start)
    
    print(f"Iteration {i} | Chunk size: {m} | % error: {error:.10f}")

In [13]:
print(f"\n📊 Average % error (continuous): {np.mean(errors):.10f}%")
print(f"📉 Min error: {np.min(errors):.10f}%, Max error: {np.max(errors):.10f}%")


📊 Average % error (continuous): 27.3951248511%
📉 Min error: 5.6903812432%, Max error: 67.7206653584%


In [ ]:
#plot graph showing percentage error for each variable against chunk size

plt.figure(figsize=(10, 6))

for var in variables:
    var_chunk_sizes = [chunk_sizes[i] for i in range(len(chunk_sizes)) if var_list[i] == var]
    var_errors = [errors[i] for i in range(len(errors)) if var_list[i] == var]
    
    plt.scatter(var_chunk_sizes, var_errors, alpha=0.6, label=var)

plt.xlabel("Chunk size")
plt.ylabel("Percentage error (%)")
plt.title("Chunk size vs. PDF % error (by variable)")
plt.grid(True)
plt.legend(title="Variable")
plt.savefig("chunk_size_vs_error_by_variable.png", dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
pdfs_by_m = {}
variable = "Income"
sd = pl.variance(ps, variable)**0.5
iqr = pl.IQR(ps, variable)["IQR"] 
global_min = ps.df[variable].min()
global_max = ps.df[variable].max() 

chunk_sizes_to_test = [5000, 50000, 100000, 250000]
pdfs_by_m = {}

# full PDF for comparison
full_grid, full_pdf = pl.PDF_or_PMF(ps, variable, var_type="continuous", plot=False)

# loop over different chunk sizes
for m in chunk_sizes_to_test:
    results = pl.process_in_chunks(
        "example.csv",
        m,
        pl.PDF_or_PMF,
        variable,
        var_type="continuous",
        sd=sd,
        iqr=iqr,
        n_samples=500000,
        global_min=global_min,
        global_max=global_max,
        grid_size=500
    )
    combined_grid, combined_pdf = pl.combine_PDF_or_PMF(results, var_type="continuous", plot=False)
    pdfs_by_m[m] = (combined_grid, combined_pdf)

# plot in grid
fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharey=True)
axes = axes.flatten()  # flatten to 1D for easy looping

for ax, (m, (grid, pdf)) in zip(axes, pdfs_by_m.items()):
    ax.plot(grid, pdf, label=f"Combined ({m})")
    ax.plot(full_grid, full_pdf, linestyle="--", color="red", label="Full PDF")
    
    pct = m / 500000 * 100  # percentage relative to 500k
    ax.set_title(f"Chunk size = {pct:.0f}%")
    
    ax.set_xlabel("Income")
    ax.grid(True)
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    if ax == axes[0]:
        ax.set_ylabel("Density")
    ax.legend()

fig.suptitle("Effect of Chunk Size on PDF Accuracy", fontsize=16, fontweight="bold")
plt.tight_layout()
plt.savefig("PDFs_computed_using_different_chunk_sizes.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#same process for EDF_or_CDF (discrete variables)

errors_edf = []
chunk_sizes_edf = []
iteration_times_edf = []
vars_list = []

for i in range(100):
    iter_start = time.time()

    discrete_var = random.choice(["Age", "CreditScore", "GenderNumeric", "Income"])
    vars_list.append(discrete_var)

    full_x, full_edf = pl.EDF_or_CDF(ps, discrete_var, var_type="discrete", plot=False)

    m = np.random.randint(1000, 250000)
    chunk_sizes_edf.append(m)

    results = pl.process_in_chunks(
        "example.csv",
        m,
        pl.EDF_or_CDF,
        discrete_var,
        var_type="discrete"
    )


    combined_x, combined_edf = pl.combine_EDF_or_CDF(results, var_type="discrete", plot=False)
    

    all_x = full_x
    full_interp = full_edf
    combined_interp = combined_edf

    error = np.mean(np.abs(full_interp - combined_interp) /
                    ((full_interp + combined_interp) / 2) * 100)
    errors_edf.append(error)
    
    iter_end = time.time()
    iteration_times_edf.append(iter_end - iter_start)
    
    print(f"Iteration {i} | Chunk size: {m} | % error: {error:.10f}")


average_edf_error = np.mean(errors_edf)
print(f"\n📊 Average % error (EDF): {average_edf_error:.10f}%")



In [37]:
print(f"\n📊 Average % error (CDF): {average_edf_error:.10f}%")
print(f"📉 Min error: {np.min(errors_edf):.15f}%, Max error: {np.max(errors_edf):.15f}%")


📊 Average % error (CDF): 0.0000000000%
📉 Min error: 0.000000000000009%, Max error: 0.000000000003261%


In [ ]:
#same process for EDF_or_CDF (continuous variables)

variables = ["Age", "Income", "CreditScore", "LoanAmount"]
errors_cdf = []
chunk_sizes_cdf = []
var_list = []

for i in range(100):

    var = random.choice(variables)
    var_list.append(var)

    global_min = ps.df[var].min()
    global_max = ps.df[var].max()

    full_x, full_cdf = pl.EDF_or_CDF(
        ps, var,
        var_type="continuous",
        plot=False
    )

    m = np.random.randint(1000, 250000)
    chunk_sizes_cdf.append(m)

    results = pl.process_in_chunks(
        "example.csv",
        m,
        pl.EDF_or_CDF,
        var,
        var_type="continuous",
        global_min=global_min,
        global_max=global_max
    )

    combined_x, combined_cdf = pl.combine_EDF_or_CDF(
        results,
        var_type="continuous",
        plot=False
    )

    error = np.mean(
        np.abs(full_cdf - combined_cdf) /
        ((full_cdf + combined_cdf) / 2) * 100
    )
    errors_cdf.append(error)

    print(f"Iteration {i} | Var: {var} | Chunk size: {m} | % error: {error:.10f}")

In [43]:
print(f"\n📊 Average % error (CDF): {np.mean(errors_cdf):.10f}%")
print(f"📉 Min error: {np.min(errors_cdf):.15f}%, Max error: {np.max(errors_cdf):.15f}%")


📊 Average % error (CDF): 0.0000000000%
📉 Min error: 0.000000000000365%, Max error: 0.000000000001406%


In [ ]:
#same process for independence (discrete variables)

errors_independence_discrete = []
chunk_sizes_independence_discrete = []
abs_errors_independence_discrete = []
variables = ["Gender", "GenderNumeric", "Smoker"]


agreements = 0  

for i in range(100):
    iter_start = time.time()
    
    x = random.choice(variables)
    y = random.choice([v for v in variables if v != x])

    res = pl.independence(ps, x, y)["p_value"]
    
    m = np.random.randint(1000, 250000)
    chunk_sizes_independence_discrete.append(m)
    
    chunk_results = pl.process_in_chunks(
        "example.csv", m, 
        pl.independence, 
        x,
        y   
    )
    combined_independence = pl.combine_independence(chunk_results)["p_value"]

    abs_error = abs(res-combined_independence)
    abs_errors_independence_discrete.append(abs_error)
    
    full_significant = res < 0.05
    chunk_significant = combined_independence < 0.05
    if full_significant == chunk_significant:
        agreements += 1
    
    print(f"Iteration {i} | Chunk size={m} | Absolute error={abs_error} | Agreement={full_significant == chunk_significant}")

In [47]:
# Final agreement percentage
agreement_percentage = agreements / 100 * 100
print(f"\nAgreement in significance decisions: {agreement_percentage:.2f}%")
print(np.mean(abs_errors_independence_discrete))


Agreement in significance decisions: 100.00%
2.887716107555227e-10


In [ ]:
#same process for independence (conitnuous variables)

errors_independence_continuous = []
chunk_sizes_independence_continuous = []
abs_errors_independence_continuous = []
variables = ["Age", "Income", "LoanAmount", "CreditScore"]


agreements = 0  

for i in range(100):
    iter_start = time.time()
    
    x = random.choice(variables)
    y = random.choice([v for v in variables if v != x])

    bin_boundaries=pl.bin_continuous_columns(ps, [x, y], return_edges=True)

    res = pl.independence(ps, x, y, bin_boundaries=bin_boundaries)["p_value"]
    
    m = np.random.randint(1000, 250000)
    chunk_sizes_independence_continuous.append(m)
    
    chunk_results = pl.process_in_chunks(
        "example.csv", m, 
        pl.independence, 
        x,
        y,
        bin_boundaries=bin_boundaries
    )
    combined_independence = pl.combine_independence(chunk_results)["p_value"]

    abs_error = abs(res-combined_independence)
    abs_errors_independence_continuous.append(abs_error)
    
    
    full_significant = res < 0.05
    chunk_significant = combined_independence < 0.05
    if full_significant == chunk_significant:
        agreements += 1
    
    print(f"Iteration {i} | Chunk size={m} | Absolute error={abs_error} | Agreement={full_significant == chunk_significant}")

In [51]:
# Final agreement percentage
agreement_percentage = agreements / 100 * 100
print(f"\nAgreement in significance decisions: {agreement_percentage:.2f}%")
print(np.mean(abs_errors_independence_continuous))


Agreement in significance decisions: 100.00%
4.3331949248744606e-17


In [ ]:
#same process for mixed pairs of variables

errors_independence_mixed = []
chunk_sizes_independence_mixed = []
abs_errors_independence_mixed = []
continuous_variables = ["Age", "Income", "LoanAmount", "CreditScore"]
discrete_variables = ["Gender", "GenderNumeric", "Smoker"]

np.random.seed(42)

agreements = 0  

for i in range(100):
    iter_start = time.time()
    
    x = random.choice(continuous_variables)
    y = random.choice(discrete_variables)

    bin_boundaries=pl.bin_continuous_columns(ps, [x], return_edges=True)

    res = pl.independence(ps, x, y, bin_boundaries=bin_boundaries)["p_value"]
    
    m = np.random.randint(1000, 250000)
    chunk_sizes_independence_continuous.append(m)
    
    chunk_results = pl.process_in_chunks(
        "example.csv", m, 
        pl.independence, 
        x,
        y,
        bin_boundaries=bin_boundaries
    )
    combined_independence = pl.combine_independence(chunk_results)["p_value"]

    abs_error = abs(res-combined_independence)
    abs_errors_independence_mixed.append(abs_error)
    
    
    # Compare significance decisions
    full_significant = res < 0.05
    chunk_significant = combined_independence < 0.05
    if full_significant == chunk_significant:
        agreements += 1
    
    print(f"Iteration {i} | Chunk size={m} | Absolute error={abs_error} | Agreement={full_significant == chunk_significant}")

In [55]:
# Final agreement percentage
agreement_percentage = agreements / 100 * 100
print(f"\nAgreement in significance decisions: {agreement_percentage:.2f}%")
print(np.mean(abs_errors_independence_mixed))


Agreement in significance decisions: 100.00%
2.0381403302479902e-10


In [ ]:
#find bin boundaries of continuous variables using chunked bin function
bin_boundaries=pl.binning_from_chunks("example.csv", 5000, continuous_columns=["Age", "CreditScore", "LoanAmount", "Income"])

In [ ]:
#run regular function 100 times to find most stable edges
n_runs = 100
edge_counter = Counter()

for i in range(n_runs):
    model, _ = pl.build_bayesian_network(
        ps,
        scoring_method='BIC',
        n_samples=None,
        scale_factor=1,
        random_state=i,
        bin_boundaries=bin_boundaries
    )
    edges = [tuple(sorted(edge)) for edge in model.edges()]
    edge_counter.update(edges)
    print(i)

#find how many iterations the edge appeared in
for edge, count in edge_counter.items():
    print(f"Edge {edge} appeared in {count}/{n_runs} runs")


In [47]:
#return list of edges which appear in all 100 iterations
stable_edges = {edge: 1 for edge, count in edge_counter.items() if count == n_runs}
stable_edges

{('Age', 'CreditScore'): 1,
 ('Age', 'Income'): 1,
 ('Income', 'LoanAmount'): 1,
 ('CreditScore', 'Income'): 1,
 ('Income', 'smoker'): 1,
 ('CreditScore', 'LoanAmount'): 1,
 ('Age', 'smoker'): 1,
 ('gender', 'gender_numeric'): 1,
 ('gender_numeric', 'smoker'): 1}

In [57]:
#list of correct edges
correct_edges = {
    ('Age', 'CreditScore'),
    ('Income', 'CreditScore'),      
    ('Age', 'Income'),
    ('Income', 'LoanAmount'),
    ('CreditScore', 'LoanAmount'),
    ('Age', 'smoker'),
    ('gender', 'gender_numeric'),  
    ('gender_numeric', 'smoker'),
}

{('CreditScore', 'LoanAmount'), ('Income', 'CreditScore'), ('Age', 'smoker'), ('Age', 'CreditScore'), ('Income', 'LoanAmount'), ('gender_numeric', 'smoker'), ('gender', 'gender_numeric'), ('Age', 'Income')}


In [85]:
#create function which compares the edges found using the chunked processor to those in the correct DAG

def score_combined_graph(learned_edges, stable_edges):
    learned_set = set(tuple(edge) for edge in learned_edges if len(edge) == 2)
    if isinstance(stable_edges, dict):
        stable_set = set(tuple(edge) for edge in stable_edges.keys() if len(edge) == 2)
    else:
        stable_set = set(tuple(edge) for edge in stable_edges if len(edge) == 2)

    matches = learned_set & stable_set
    reversed_edges = set((v, u) for (u, v) in learned_set if (v, u) in stable_set)
    missing_edges = stable_set - learned_set - reversed_edges
    extra_edges = learned_set - stable_set - reversed_edges

    accuracy = len(matches) / len(stable_set) if stable_set else 0

    return {
        "accuracy": accuracy,
        "matches": matches,
        "reversed_edges": reversed_edges,
        "missing_edges": missing_edges,
        "extra_edges": extra_edges,
        "total_edges": len(stable_set)
    }

In [ ]:
#measure accuracy of combine_bayesian_models when using the hybrid method
accuracy = []
chunk_size = []
sample_size = []
incorrect_edges = []
reversed_edges = []
related_edges = []
total_edges_list = []

start_time = time.time()

for i in range(100):
    m = np.random.randint(1000, 250000)
    y = np.random.random()

    results = pl.process_in_chunks(
        "example.csv",
        func=pl.build_bayesian_network_edges,
        chunk_size=m,
        sample_size=int(m*y),
        bin_boundaries=bin_boundaries,
        scoring_method="BIC"
    )

    sorted_edges = pl.combine_bayesian_models(results, top_k=8)

    dict_scores = score_combined_graph(sorted_edges, correct_edges)

    accuracy.append(dict_scores["accuracy"])
    chunk_size.append(m)
    sample_size.append(y)
    incorrect_edges.append(len(dict_scores["missing_edges"]) + len(dict_scores["extra_edges"]))
    reversed_edges.append(len(dict_scores["reversed_edges"]))
    related_edges.append(len(dict_scores["reversed_edges"]) + len(dict_scores["matches"]))
    total_edges_list.append(dict_scores["total_edges"])

    print(f"Iteration {i} completed")

end_time = time.time()
elapsed = end_time - start_time
print(f"⏱️ Total processing time: {elapsed:.2f} seconds")

In [69]:
df = pd.DataFrame({
    "chunk_size": chunk_size,
    "sample_fraction": sample_size,
    "accuracy": accuracy,
    "incorrect_edges": incorrect_edges,
    "reversed_edges": reversed_edges,
    "related_edges": related_edges,
    "total_edges": total_edges_list
})

correct_edges_abs = np.array(accuracy) * np.array(total_edges_list)

avg_correct_edges = correct_edges_abs.mean()

print(f"Average number of correctly learned edges (excluding reversed): {avg_correct_edges:.2f}")
df.describe()

Average number of correctly learned edges (excluding reversed): 3.82


,chunk_size,sample_fraction,accuracy,incorrect_edges,reversed_edges,related_edges,total_edges
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0
mean,126012.190000,0.524294,0.477500,6.590000,3.990000,7.810000,8.0
std,70892.643964,0.293125,0.080206,1.223466,0.688946,0.442559,0.0
min,3568.000000,0.006952,0.375000,4.000000,2.000000,6.000000,8.0
25%,69976.500000,0.275032,0.375000,6.000000,4.000000,8.000000,8.0
50%,125236.000000,0.536716,0.500000,7.000000,4.000000,8.000000,8.0
75%,189421.750000,0.779733,0.500000,7.000000,4.000000,8.000000,8.0
max,248255.000000,0.997740,0.750000,9.000000,5.000000,8.000000,8.0


In [ ]:
plt.scatter(chunk_size, accuracy)
plt.xlabel("Chunk size")
plt.ylabel("Accuracy (%)")
plt.title("Chunk size vs Accuracy")
plt.savefig("chunk_size_vs_accuracy_hybrid.png", dpi=300, bbox_inches='tight')
plt.show()

plt.scatter(sample_size, accuracy)
plt.xlabel("Sample size fraction")
plt.ylabel("Accuracy (%)")
plt.title("Sample size vs Accuracy")
plt.savefig("sample_size_vs_accuracy_hybrid.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#same process when using frequency method
accuracy_frequency = []
chunk_size_frequency = []
sample_size_frequency = []
incorrect_edges_frequency = []
reversed_edges_frequency = []
related_edges_frequency = []
total_edges_list_frequency = []


for i in range(100):
    m = np.random.randint(1000, 250000)
    y = np.random.random()

    results = pl.process_in_chunks(
        "example.csv",
        func=pl.build_bayesian_network_edges,
        chunk_size=m,
        sample_size=int(m*y),
        bin_boundaries=bin_boundaries,
        scoring_method="BIC"
    )

    sorted_edges = pl.combine_bayesian_models(results, top_k=8, score_type="frequency")

    dict_scores = score_combined_graph(sorted_edges, correct_edges)

    accuracy_frequency.append(dict_scores["accuracy"])
    chunk_size_frequency.append(m)
    sample_size_frequency.append(y)
    incorrect_edges_frequency.append(len(dict_scores["missing_edges"]) + len(dict_scores["extra_edges"]))
    reversed_edges_frequency.append(len(dict_scores["reversed_edges"]))
    related_edges_frequency.append(len(dict_scores["reversed_edges"]) + len(dict_scores["matches"]))
    total_edges_list_frequency.append(dict_scores["total_edges"])

    print(f"Iteration {i} completed")

end_time = time.time()
elapsed = end_time - start_time
print(f"⏱️ Total processing time: {elapsed:.2f} seconds")


In [75]:
df_frequency = pd.DataFrame({
    "chunk_size": chunk_size_frequency,
    "sample_fraction": sample_size_frequency,
    "accuracy": accuracy_frequency,
    "incorrect_edges": incorrect_edges_frequency,
    "reversed_edges": reversed_edges_frequency,
    "related_edges": related_edges_frequency,
    "total_edges": total_edges_list_frequency
})

correct_edges_abs_frequency = np.array(accuracy_frequency) * np.array(total_edges_list_frequency)

avg_correct_edges_frequency = correct_edges_abs_frequency.mean()

print(f"Average number of correctly learned edges (excluding reversed): {avg_correct_edges_frequency:.2f}")
df_frequency.describe()

Average number of correctly learned edges (excluding reversed): 3.63


,chunk_size,sample_fraction,accuracy,incorrect_edges,reversed_edges,related_edges,total_edges
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0
mean,131280.150000,0.516096,0.453750,5.160000,3.450000,7.080000,8.0
std,66449.165036,0.270651,0.098369,1.220366,0.783349,0.691653,0.0
min,4420.000000,0.014393,0.250000,2.000000,2.000000,5.000000,8.0
25%,85397.750000,0.334235,0.375000,4.000000,3.000000,7.000000,8.0
50%,131828.000000,0.520286,0.500000,5.000000,3.000000,7.000000,8.0
75%,189151.500000,0.703703,0.500000,6.000000,4.000000,7.250000,8.0
max,249710.000000,0.994457,0.750000,9.000000,5.000000,8.000000,8.0


In [ ]:
plt.scatter(chunk_size_frequency, accuracy_frequency)
plt.xlabel("Chunk size")
plt.ylabel("Accuracy (%)")
plt.title("Chunk size vs Accuracy")
plt.savefig("chunk_size_vs_accuracy_frequency.png", dpi=300, bbox_inches='tight')
plt.show()

plt.scatter(sample_size_frequency, accuracy_frequency)
plt.xlabel("Sample size fraction")
plt.ylabel("Accuracy (%)")
plt.title("Sample size vs Accuracy")
plt.savefig("sample_size_vs_accuracy_hybrid.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#same for softmax method
accuracy_softmax = []
chunk_size_softmax = []
sample_size_softmax = []
incorrect_edges_softmax = []
reversed_edges_softmax = []
related_edges_softmax = []
total_edges_list_softmax = []

start_time = time.time()

for i in range(100):
    m = np.random.randint(1000, 250000)
    y = np.random.random()

    results = pl.process_in_chunks(
        "example.csv",
        func=pl.build_bayesian_network_edges,
        chunk_size=m,
        sample_size=int(m*y),
        bin_boundaries=bin_boundaries,
        handle_missing="drop",
        scoring_method="BIC"
    )

    sorted_edges = pl.combine_bayesian_models(results, top_k=8, score_type="softmax")

    dict_scores = score_combined_graph(sorted_edges, correct_edges)

    accuracy_softmax.append(dict_scores["accuracy"])
    chunk_size_softmax.append(m)
    sample_size_softmax.append(y)
    incorrect_edges_softmax.append(len(dict_scores["missing_edges"]) + len(dict_scores["extra_edges"]))
    reversed_edges_softmax.append(len(dict_scores["reversed_edges"]))
    related_edges_softmax.append(len(dict_scores["reversed_edges"]) + len(dict_scores["matches"]))
    total_edges_list_softmax.append(dict_scores["total_edges"])

    print(f"Iteration {i} completed")

end_time = time.time()
elapsed = end_time - start_time
print(f"⏱️ Total processing time: {elapsed:.2f} seconds")

In [81]:
df_softmax = pd.DataFrame({
    "chunk_size": chunk_size_softmax,
    "sample_fraction": sample_size_softmax,
    "accuracy": accuracy_softmax,
    "incorrect_edges": incorrect_edges_softmax,
    "reversed_edges": reversed_edges_softmax,
    "related_edges": related_edges_softmax,
    "total_edges": total_edges_list_softmax
})

correct_edges_abs_softmax = np.array(accuracy_softmax) * np.array(total_edges_list_softmax)

avg_correct_edges_softmax= correct_edges_abs_softmax.mean()

print(f"Average number of correctly learned edges (excluding reversed): {avg_correct_edges_softmax:.2f}")
print(df_softmax.describe())

matches_list = [related - reversed_ for related, reversed_ in zip(related_edges_softmax, reversed_edges_softmax)]

max_correct_edges = max(matches_list)
print(f"Maximum correct edges (not reversed) = {max_correct_edges}")

Average number of correctly learned edges (excluding reversed): 2.77
          chunk_size  sample_fraction   accuracy  incorrect_edges  \
count     100.000000       100.000000  100.00000       100.000000   
mean   116187.760000         0.467512    0.34625         7.640000   
std     74778.218621         0.295838    0.14308         2.560461   
min      2062.000000         0.010838    0.12500         3.000000   
25%     53534.500000         0.233158    0.25000         6.000000   
50%    108415.500000         0.420857    0.37500         8.000000   
75%    179809.750000         0.732995    0.50000         9.250000   
max    242080.000000         0.995831    0.62500        13.000000   

       reversed_edges  related_edges  total_edges  
count       100.00000     100.000000        100.0  
mean          2.14000       4.910000          8.0  
std           0.85304       1.443166          0.0  
min           0.00000       2.000000          8.0  
25%           2.00000       4.000000          8.0

In [ ]:
plt.scatter(chunk_size_softmax, accuracy_softmax)
plt.xlabel("Chunk size")
plt.ylabel("Accuracy (%)")
plt.title("Chunk size vs Accuracy")
plt.savefig("chunk_size_vs_accuracy_softmax.png", dpi=300, bbox_inches='tight')
plt.show()

plt.scatter(sample_size_softmax, accuracy_softmax)
plt.xlabel("Sample size fraction")
plt.ylabel("Accuracy (%)")
plt.title("Sample size vs Accuracy")
plt.savefig("sample_size_vs_accuracy_softmax.png", dpi=300, bbox_inches='tight')
plt.show()

In [87]:
#new function to compare correct edges to those returned by aggregate_and_build_network

def score_edges(predicted_edges, correct_edges):

    predicted_set = set(predicted_edges)
    correct_set = set(correct_edges)
    
    matches = len(predicted_set & correct_set)
    
    reversed_edges = len([e for e in predicted_set if (e[1], e[0]) in correct_set])
    
    missing_edges = len(correct_set - predicted_set - {(e[1], e[0]) for e in predicted_set})
    
    extra_edges = len(predicted_set - correct_set - {(e[1], e[0]) for e in correct_set})
    
    accuracy = 100 * matches / len(correct_set) if correct_set else 0.0

    accuracy_with_reversals = 100 * (matches + reversed_edges) / len(correct_set) if correct_set else 0.0
    
    return {
        'matches': matches,
        'reversed_edges': reversed_edges,
        'missing_edges': missing_edges,
        'extra_edges': extra_edges,
        'accuracy': accuracy,
        'accuracy_with_reversals': accuracy_with_reversals
    }

In [ ]:
#measure accuracy of aggregate_and_build_network
accuracy_3 = []
chunk_size_3 = []
sample_size_3 = []
incorrect_edges_3 = []
missing_edges_3 = []
extra_edges_3 = []
reversed_edges_3 = []
accuracy_with_reversals_3 = []
related_edges_3 = []
start_time = time.time()

for i in range(50):
    m = np.random.randint(1000, 250000)
    y = np.random.random()

    results = pl.process_in_chunks(
        "example.csv",
        func=pl.build_bayesian_network,
        chunk_size=m,
        sample_size=int(m*y),
        handle_missing="drop",
        scoring_method="BIC"
    )

      
    consensus_edges, edge_scores = pl.aggregate_and_build_network(results, bin_boundaries=bin_boundaries)

    dict = score_edges(consensus_edges, correct_edges)

    accuracy_3.append(dict["accuracy"])
    chunk_size_3.append(m)
    sample_size_3.append(y)
    missing_edges_3.append(dict["missing_edges"])
    extra_edges_3.append(dict["missing_edges"])
    incorrect_edges_3.append(dict["missing_edges"]+dict["extra_edges"])
    reversed_edges_3.append(dict["reversed_edges"])
    related_edges_3.append(dict["reversed_edges"]+dict["matches"])
    accuracy_with_reversals_3.append(dict["accuracy_with_reversals"])
    
    
    print(f"Iteration {i} completed")

end_time = time.time()
elapsed = end_time - start_time

In [ ]:
#analysis of results
print(np.mean(accuracy_3))
print(np.mean(incorrect_edges_3))
print(incorrect_edges_3)
print(missing_edges_3)
print(np.mean(accuracy_with_reversals_3))
print(np.max(accuracy_with_reversals_3))
print(np.mean(missing_edges_3))
print(related_edges)
print(np.mean(related_edges))

In [101]:
#time elapsed
elapsed

4650.428697347641

In [ ]:
plt.scatter(chunk_size_3, accuracy_3)
plt.xlabel("Chunk size")
plt.ylabel("Accuracy (%)")
plt.title("Chunk size vs Accuracy")
plt.savefig("chunk_size_vs_accuracy_bayesian_netowrk.png", dpi=300, bbox_inches='tight')
plt.show()

plt.scatter(sample_size_3, accuracy_with_reversals_3)
plt.xlabel("Sample size fraction")
plt.ylabel("Accuracy (%)")
plt.title("Sample size vs Accuracy (related edges included)")
plt.savefig("related_edges_vs_chunk_size_bayesian_netowrk.png", dpi=300, bbox_inches='tight')
plt.show()

plt.scatter(sample_size_3, accuracy_3)
plt.xlabel("Sample size fraction")
plt.ylabel("Accuracy (%)")
plt.title("Sample size vs Accuracy")
plt.savefig("Sample_size_vs_Accuracy_bayesian_netowrk.png", dpi=300, bbox_inches='tight')
plt.show()